# Simulation 04 -- Cluster Quality Stress Test

**Question:** What happens when clusters are too coarse, too fine, or when important features are missing?

### Why This Matters

Our pipeline's first stage groups sites into **peer clusters** using HDBSCAN. Everything downstream depends on cluster quality:
- **Too coarse** (few large clusters) → heterogeneous peer groups → noisy pseudo-labels → biased FMV
- **Too fine** (many tiny clusters) → insufficient samples per cluster → unstable MAD statistics → noisy pseudo-labels
- **Wrong features** → clusters that don't reflect market similarity → pseudo-labels that mix incomparable sites

### What We Test

| Part | Test | Question |
|:---|:---|:---|
| **A** | Cluster Granularity | What `min_cluster_size` minimizes final MAPE? |
| **B** | Missing Features | How much does each feature group contribute? |
| **C** | K-Means vs HDBSCAN | Is density-based clustering actually better? |

In [ ]:
# ============================================================
# Imports & Setup
# ============================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, time, warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import Ridge
from sklearn.cluster import KMeans
from sklearn.metrics import (
    silhouette_score, mean_absolute_error, r2_score,
    confusion_matrix, classification_report, cohen_kappa_score
)
from scipy.stats import median_abs_deviation
from xgboost import XGBRegressor

# HDBSCAN: try standalone package first, fall back to sklearn
try:
    from hdbscan import HDBSCAN
    print('Using standalone hdbscan package')
except ImportError:
    from sklearn.cluster import HDBSCAN
    print('Using sklearn HDBSCAN')

# Quantile forest: try quantile-forest package, fall back to sklearn RF
try:
    from quantile_forest import RandomForestQuantileRegressor
    HAS_QRF = True
    print('Using quantile-forest package')
except ImportError:
    from sklearn.ensemble import RandomForestRegressor
    HAS_QRF = False
    print('Falling back to sklearn RandomForestRegressor (no native quantiles)')

np.random.seed(42)

# Plot style
sns.set_style('whitegrid')
plt.rcParams.update({
    'figure.dpi': 120,
    'font.size': 10,
    'axes.titlesize': 12,
    'axes.labelsize': 10,
})

print('All imports successful.')

## 1. Load Synthetic Data & Define Feature Sets

In [ ]:
# ============================================================
# Load the synthetic dataset generated in sim_00
# ============================================================
DATA_PATH = os.path.join('..', 'data', 'synthetic', 'synthetic_5k_sites.parquet')
df = pd.read_parquet(DATA_PATH)
print(f'Loaded {len(df):,} sites, {df.shape[1]} columns')
print(f'True FMV range: ${df["true_fmv"].min():,.0f} - ${df["true_fmv"].max():,.0f}')
print(f'True FMV median: ${df["true_fmv"].median():,.0f}')

# ============================================================
# Feature lists
# ============================================================
X_MARKET = [
    'census_population_density', 'census_population_3km',
    'census_median_income', 'census_median_home_value',
    'census_median_rent', 'census_vacancy_rate',
    'urban_rural_class', 'underlying_land_value_psf',
    'hud_safmr', 'property_tax_rate',
    'topography_class', 'terrain_elevation_variance_m',
    'poi_density_1km', 'building_density_500m',
    'distance_to_highway_km', 'ground_elevation_ft',
    'unemployment_rate_local',
    'fcc_tower_count_5km', 'fcc_tower_density_sqkm',
    'competitor_tower_distance_km', 'tower_per_capita',
    'scarcity_index', 'landlord_concentration',
    'composite_hazard_score',
    'tower_height_ft', 'lease_area_sqft', 'parcel_acreage',
    'construction_age', 'site_access_ordinal',
    'permit_approval_difficulty', 'wrluri_index',
    'colocation_required', 'height_restriction_ft',
    'has_building', 'is_ground_tower', 'has_parcel', 'has_zoning_data',
]

Z_NETWORK = [
    'tenant_count', 'primary_technology', 'coverage_critical',
    'capacity_utilization_pct', 'high_traffic',
    'backhaul_type', 'has_fiber_connection',
    'network_substitutability_score', 'search_ring_alternative_count',
    'inter_site_distance_km', 'antenna_count',
    'structure_capacity_remaining', 'site_priority_tier',
]

ALL_FEATURES = X_MARKET + Z_NETWORK

# Ground truth
true_fmv = df['true_fmv'].values
rents = df['current_monthly_rent'].values

print(f'\nX_MARKET features: {len(X_MARKET)}')
print(f'Z_NETWORK features: {len(Z_NETWORK)}')
print(f'Total features:     {len(ALL_FEATURES)}')

# Verify
missing = [f for f in ALL_FEATURES if f not in df.columns]
assert len(missing) == 0, f'Missing: {missing}'
print('All features verified. \u2713')

## 2. Reusable Pipeline Function

A configurable pipeline that accepts different clustering parameters, feature subsets, and clustering algorithms.

In [ ]:
def run_pipeline(df, true_fmv, x_features=None, z_features=None,
                 cluster_method='hdbscan', min_cluster_size=20,
                 kmeans_k=None, mad_threshold=1.0, label='pipeline'):
    """
    Run the full FMV estimation pipeline with configurable clustering.
    
    Parameters
    ----------
    df : pd.DataFrame
        Dataset with features and current_monthly_rent.
    true_fmv : array-like
        Ground truth FMV for evaluation.
    x_features : list, optional
        Market features for clustering. Defaults to X_MARKET.
    z_features : list, optional
        Network features for supervised model. Defaults to Z_NETWORK.
    cluster_method : str
        'hdbscan' or 'kmeans'.
    min_cluster_size : int
        HDBSCAN min_cluster_size parameter.
    kmeans_k : int, optional
        Number of clusters for K-Means.
    mad_threshold : float
        MAD threshold for CORE labeling.
    label : str
        Label for logging.
    
    Returns
    -------
    dict with cluster metrics, model predictions, error metrics, and timing.
    """
    if x_features is None:
        x_features = X_MARKET
    if z_features is None:
        z_features = Z_NETWORK
    
    all_feats = x_features + z_features
    rent_vals = df['current_monthly_rent'].values.copy()
    n = len(df)
    timings = {}
    
    # ---- Stage 1: Clustering ----
    t0 = time.time()
    scaler_clust = StandardScaler()
    X_clust = scaler_clust.fit_transform(df[x_features].values)
    
    if cluster_method == 'hdbscan':
        clusterer = HDBSCAN(min_cluster_size=min_cluster_size, min_samples=5)
        cluster_ids = clusterer.fit_predict(X_clust)
    elif cluster_method == 'kmeans':
        clusterer = KMeans(n_clusters=kmeans_k, random_state=42, n_init=10)
        cluster_ids = clusterer.fit_predict(X_clust)
    else:
        raise ValueError(f'Unknown method: {cluster_method}')
    
    n_clusters = len(set(cluster_ids) - {-1})
    noise_count = (cluster_ids == -1).sum()
    noise_pct = noise_count / n
    
    # Assign noise points to nearest cluster (HDBSCAN only)
    cluster_ids_clean = cluster_ids.copy()
    if noise_count > 0:
        from sklearn.neighbors import NearestNeighbors
        valid_mask = cluster_ids_clean != -1
        if valid_mask.sum() > 0:
            nn = NearestNeighbors(n_neighbors=1).fit(X_clust[valid_mask])
            noise_mask = cluster_ids_clean == -1
            _, idx = nn.kneighbors(X_clust[noise_mask])
            cluster_ids_clean[noise_mask] = cluster_ids_clean[valid_mask][idx.ravel()]
    
    # Silhouette score (sample for speed if >2000 points)
    try:
        if n > 2000:
            sample_idx = np.random.choice(n, 2000, replace=False)
            sil = silhouette_score(X_clust[sample_idx], cluster_ids_clean[sample_idx])
        else:
            sil = silhouette_score(X_clust, cluster_ids_clean)
    except Exception:
        sil = -1  # if only 1 cluster
    
    # Within-cluster rent CV (coefficient of variation)
    cluster_cvs = []
    for cid in np.unique(cluster_ids_clean):
        c_rents = rent_vals[cluster_ids_clean == cid]
        if len(c_rents) >= 3 and np.mean(c_rents) > 0:
            cv = np.std(c_rents) / np.mean(c_rents)
            cluster_cvs.append(cv)
    mean_cv = np.mean(cluster_cvs) if cluster_cvs else np.nan
    
    timings['clustering'] = time.time() - t0
    
    # ---- Stage 2: MAD Pseudo-Labeling ----
    t0 = time.time()
    pseudo_labels = np.full(n, 'EDGE', dtype='U12')
    
    for cid in np.unique(cluster_ids_clean):
        mask = cluster_ids_clean == cid
        c_rents = rent_vals[mask]
        med = np.median(c_rents)
        mad = median_abs_deviation(c_rents, scale='normal')
        if mad < 1e-6:
            mad = np.std(c_rents) * 0.6745
        
        dev = np.abs(c_rents - med) / mad
        core = dev <= mad_threshold
        outlier = dev > 2.0
        
        idx = np.where(mask)[0]
        pseudo_labels[idx[core]] = 'CORE'
        pseudo_labels[idx[outlier]] = 'OUTLIER'
    
    core_pct = (pseudo_labels == 'CORE').mean()
    timings['pseudo_labeling'] = time.time() - t0
    
    # ---- Stage 3: Supervised Model on CORE sites ----
    t0 = time.time()
    core_mask = pseudo_labels == 'CORE'
    
    if core_mask.sum() < 20:
        # Fallback: if too few CORE, use all non-OUTLIER
        core_mask = pseudo_labels != 'OUTLIER'
    
    X_all = df[all_feats].values
    X_train = X_all[core_mask]
    y_train = rent_vals[core_mask]
    
    # XGBoost
    xgb = XGBRegressor(
        n_estimators=500, learning_rate=0.05, max_depth=6,
        subsample=0.8, colsample_bytree=0.8,
        min_child_weight=5, reg_alpha=0.1, reg_lambda=1.0,
        random_state=42, n_jobs=-1
    )
    xgb.fit(X_train, y_train)
    pred_xgb = xgb.predict(X_all)
    
    # QRF
    if HAS_QRF:
        qrf = RandomForestQuantileRegressor(
            n_estimators=500, min_samples_leaf=5,
            random_state=42, n_jobs=-1
        )
        qrf.fit(X_train, y_train)
        pred_q50 = qrf.predict(X_all, quantiles=0.50)
    else:
        rf = RandomForestRegressor(
            n_estimators=500, min_samples_leaf=5,
            random_state=42, n_jobs=-1
        )
        rf.fit(X_train, y_train)
        pred_q50 = rf.predict(X_all)
    
    # Meta-learner
    meta_X = np.column_stack([pred_xgb, pred_q50])
    ridge = Ridge(alpha=1.0)
    ridge.fit(meta_X[core_mask], y_train)
    predicted_fmv = np.maximum(ridge.predict(meta_X), 100)
    
    timings['modeling'] = time.time() - t0
    
    # ---- Evaluation vs true FMV ----
    mape = np.mean(np.abs(predicted_fmv - true_fmv) / true_fmv) * 100
    mae = mean_absolute_error(true_fmv, predicted_fmv)
    r2 = r2_score(true_fmv, predicted_fmv)
    bias = np.mean(predicted_fmv - true_fmv)
    
    return {
        'label': label,
        'predicted_fmv': predicted_fmv,
        'n_clusters': n_clusters,
        'noise_pct': noise_pct,
        'silhouette': sil,
        'mean_rent_cv': mean_cv,
        'core_pct': core_pct,
        'mape': mape,
        'mae': mae,
        'r2': r2,
        'bias': bias,
        'timings': timings,
        'cluster_ids': cluster_ids_clean,
    }


print('Pipeline function defined.')

---

## Part A: Cluster Granularity Test

HDBSCAN's `min_cluster_size` controls the granularity of clustering:
- **Small values** (10-20): many small, tight clusters — better homogeneity but potentially unstable MAD estimates
- **Large values** (200-500): fewer large clusters — more samples per cluster but higher within-cluster variance

We sweep across 6 values and measure the impact on the full pipeline.

In [ ]:
# ============================================================
# Part A: Run pipeline with different min_cluster_size values
# ============================================================
MIN_CLUSTER_SIZES = [10, 20, 50, 100, 200, 500]

granularity_results = []

for mcs in MIN_CLUSTER_SIZES:
    print(f'\nRunning min_cluster_size = {mcs}...')
    t_start = time.time()
    
    res = run_pipeline(
        df, true_fmv,
        cluster_method='hdbscan',
        min_cluster_size=mcs,
        label=f'HDBSCAN(mcs={mcs})'
    )
    
    t_total = time.time() - t_start
    
    granularity_results.append({
        'min_cluster_size': mcs,
        'n_clusters': res['n_clusters'],
        'noise_pct': res['noise_pct'] * 100,
        'silhouette': res['silhouette'],
        'mean_rent_cv': res['mean_rent_cv'],
        'core_pct': res['core_pct'] * 100,
        'mape': res['mape'],
        'mae': res['mae'],
        'r2': res['r2'],
        'bias': res['bias'],
        'time_s': t_total,
    })
    
    print(f'  Clusters: {res["n_clusters"]}, Noise: {res["noise_pct"]*100:.1f}%, '
          f'Silhouette: {res["silhouette"]:.3f}')
    print(f'  Within-cluster rent CV: {res["mean_rent_cv"]:.3f}, '
          f'Core: {res["core_pct"]*100:.1f}%')
    print(f'  MAPE: {res["mape"]:.1f}%, MAE: ${res["mae"]:,.0f}, '
          f'R2: {res["r2"]:.3f}, Bias: ${res["bias"]:+,.0f}')
    print(f'  Time: {t_total:.1f}s')

gran_df = pd.DataFrame(granularity_results)
print('\n' + '='*80)
print('GRANULARITY TEST RESULTS')
print('='*80)
print(gran_df.to_string(index=False))

In [ ]:
# ============================================================
# Part A Plots: Granularity Impact
# ============================================================
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

mcs_vals = gran_df['min_cluster_size'].values

# (0,0) MAPE vs min_cluster_size
ax = axes[0, 0]
ax.plot(mcs_vals, gran_df['mape'], 'ro-', ms=10, lw=2)
best_idx = gran_df['mape'].idxmin()
ax.scatter([mcs_vals[best_idx]], [gran_df['mape'].iloc[best_idx]],
           s=200, c='gold', edgecolors='black', zorder=5, marker='*',
           label=f'Best: mcs={mcs_vals[best_idx]}')
ax.set_xlabel('min_cluster_size')
ax.set_ylabel('MAPE (%)')
ax.set_title('MAPE vs Cluster Granularity', fontweight='bold')
ax.axhline(15, color='green', ls='--', alpha=0.4, label='15% target')
ax.legend(fontsize=9)
ax.set_xscale('log')

# (0,1) Silhouette vs min_cluster_size
ax = axes[0, 1]
ax.plot(mcs_vals, gran_df['silhouette'], 'bs-', ms=10, lw=2)
ax.set_xlabel('min_cluster_size')
ax.set_ylabel('Silhouette Score')
ax.set_title('Silhouette vs Cluster Granularity', fontweight='bold')
ax.axhline(0.30, color='green', ls='--', alpha=0.4, label='0.30 target')
ax.legend(fontsize=9)
ax.set_xscale('log')

# (0,2) Number of clusters
ax = axes[0, 2]
ax.bar([str(x) for x in mcs_vals], gran_df['n_clusters'],
       color='steelblue', edgecolor='white')
for i, (v, nc) in enumerate(zip(mcs_vals, gran_df['n_clusters'])):
    ax.text(i, nc + 0.5, str(nc), ha='center', fontsize=10, fontweight='bold')
ax.set_xlabel('min_cluster_size')
ax.set_ylabel('Number of Clusters')
ax.set_title('Cluster Count by Granularity', fontweight='bold')

# (1,0) Noise percentage
ax = axes[1, 0]
ax.plot(mcs_vals, gran_df['noise_pct'], 'g^-', ms=10, lw=2)
ax.set_xlabel('min_cluster_size')
ax.set_ylabel('Noise Points (%)')
ax.set_title('Noise Percentage vs Granularity', fontweight='bold')
ax.set_xscale('log')

# (1,1) Within-cluster rent CV
ax = axes[1, 1]
ax.plot(mcs_vals, gran_df['mean_rent_cv'], 'mD-', ms=10, lw=2)
ax.set_xlabel('min_cluster_size')
ax.set_ylabel('Mean Within-Cluster Rent CV')
ax.set_title('Cluster Homogeneity (lower = better)', fontweight='bold')
ax.set_xscale('log')

# (1,2) R² score
ax = axes[1, 2]
ax.plot(mcs_vals, gran_df['r2'], 'co-', ms=10, lw=2)
ax.set_xlabel('min_cluster_size')
ax.set_ylabel('R² Score')
ax.set_title('R² vs Cluster Granularity', fontweight='bold')
ax.set_xscale('log')

plt.suptitle('Part A: Cluster Granularity Test', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Report optimal
opt = gran_df.loc[gran_df['mape'].idxmin()]
print(f'\nOptimal min_cluster_size = {int(opt["min_cluster_size"])} '
      f'(MAPE={opt["mape"]:.1f}%, Silhouette={opt["silhouette"]:.3f}, '
      f'Clusters={int(opt["n_clusters"])}, Rent CV={opt["mean_rent_cv"]:.3f})')

---

## Part B: Missing Feature Test

We systematically drop individual features or feature groups to measure their importance to the pipeline.

| Scenario | What's Dropped | Why It Matters |
|:---|:---|:---|
| Drop `census_median_home_value` | Strongest market signal in the FMV equation | Tests robustness to missing key predictor |
| Drop `tenant_count` | Primary network signal | Tests whether network data adds value |
| Drop `scarcity_index` | Competition/supply metric | Tests competition feature importance |
| Drop ALL zoning features | 5 regulatory features | Tests zoning contribution |
| Drop ALL Z_NETWORK | 13 network features | Tests whether network features matter at all |

In [ ]:
# ============================================================
# Part B: Define drop scenarios
# ============================================================
ZONING_FEATURES = [
    'permit_approval_difficulty', 'wrluri_index',
    'colocation_required', 'height_restriction_ft', 'has_zoning_data'
]

DROP_SCENARIOS = {
    'Baseline (all features)': {
        'x_drop': [],
        'z_drop': [],
    },
    'Drop census_median_home_value': {
        'x_drop': ['census_median_home_value'],
        'z_drop': [],
    },
    'Drop tenant_count': {
        'x_drop': [],
        'z_drop': ['tenant_count'],
    },
    'Drop scarcity_index': {
        'x_drop': ['scarcity_index'],
        'z_drop': [],
    },
    'Drop ALL zoning': {
        'x_drop': ZONING_FEATURES,
        'z_drop': [],
    },
    'Drop ALL Z_NETWORK': {
        'x_drop': [],
        'z_drop': Z_NETWORK.copy(),  # drop all network features from supervised model
    },
}

print(f'Defined {len(DROP_SCENARIOS)} drop scenarios')
for name, spec in DROP_SCENARIOS.items():
    total_drop = len(spec['x_drop']) + len(spec['z_drop'])
    print(f'  {name}: dropping {total_drop} features')

In [ ]:
# ============================================================
# Part B: Run pipeline for each drop scenario
# ============================================================
drop_results = []

for name, spec in DROP_SCENARIOS.items():
    print(f'\nScenario: {name}')
    
    # Build feature lists with drops
    x_feats = [f for f in X_MARKET if f not in spec['x_drop']]
    z_feats = [f for f in Z_NETWORK if f not in spec['z_drop']]
    
    print(f'  X_MARKET: {len(x_feats)} features (dropped {len(spec["x_drop"])})')
    print(f'  Z_NETWORK: {len(z_feats)} features (dropped {len(spec["z_drop"])})')
    
    res = run_pipeline(
        df, true_fmv,
        x_features=x_feats,
        z_features=z_feats,
        cluster_method='hdbscan',
        min_cluster_size=20,
        label=name
    )
    
    drop_results.append({
        'Scenario': name,
        'X_features': len(x_feats),
        'Z_features': len(z_feats),
        'Clusters': res['n_clusters'],
        'Silhouette': res['silhouette'],
        'Rent CV': res['mean_rent_cv'],
        'Core %': res['core_pct'] * 100,
        'MAPE': res['mape'],
        'MAE': res['mae'],
        'R2': res['r2'],
        'Bias ($)': res['bias'],
    })
    
    print(f'  MAPE: {res["mape"]:.1f}%, MAE: ${res["mae"]:,.0f}, R2: {res["r2"]:.3f}')

drop_df = pd.DataFrame(drop_results)

# Compute MAPE increase vs baseline
baseline_mape = drop_df.loc[drop_df['Scenario'] == 'Baseline (all features)', 'MAPE'].values[0]
drop_df['MAPE Increase'] = drop_df['MAPE'] - baseline_mape
drop_df['MAPE Increase (%)'] = (drop_df['MAPE Increase'] / baseline_mape * 100)

print('\n' + '='*90)
print('MISSING FEATURE TEST RESULTS')
print('='*90)
display_cols = ['Scenario', 'X_features', 'Z_features', 'Clusters',
                'Silhouette', 'MAPE', 'MAPE Increase', 'R2', 'Bias ($)']
print(drop_df[display_cols].to_string(index=False))

In [ ]:
# ============================================================
# Part B Plot: MAPE increase per drop scenario
# ============================================================
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart of MAPE increase
ax = axes[0]
non_baseline = drop_df[drop_df['Scenario'] != 'Baseline (all features)'].copy()
non_baseline = non_baseline.sort_values('MAPE Increase', ascending=True)

colors = ['#e74c3c' if x > 2 else '#f39c12' if x > 0.5 else '#2ecc71'
          for x in non_baseline['MAPE Increase']]
bars = ax.barh(range(len(non_baseline)), non_baseline['MAPE Increase'],
               color=colors, edgecolor='white', height=0.6)

ax.set_yticks(range(len(non_baseline)))
ax.set_yticklabels(non_baseline['Scenario'].values, fontsize=9)
for i, (val, mape_val) in enumerate(zip(non_baseline['MAPE Increase'], non_baseline['MAPE'])):
    sign = '+' if val >= 0 else ''
    ax.text(max(val, 0) + 0.1, i, f'{sign}{val:.1f}pp (MAPE={mape_val:.1f}%)',
            va='center', fontsize=9)
ax.set_xlabel('MAPE Increase (percentage points)')
ax.set_title('Feature Importance: MAPE Impact of Dropping Features', fontweight='bold')
ax.axvline(0, color='black', ls='-', lw=0.5)

# Grouped bar: MAPE comparison
ax = axes[1]
scenarios = drop_df['Scenario'].values
short_names = ['Baseline', 'Drop Home\nValue', 'Drop Tenant\nCount',
               'Drop Scarcity\nIndex', 'Drop ALL\nZoning', 'Drop ALL\nZ_NETWORK']
mapes = drop_df['MAPE'].values

bar_colors = ['#2ecc71'] + ['#e74c3c' if m > baseline_mape + 1 else '#f39c12'
                             for m in mapes[1:]]
bars = ax.bar(short_names, mapes, color=bar_colors, edgecolor='white', width=0.7)
for bar, val in zip(bars, mapes):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2,
            f'{val:.1f}%', ha='center', fontsize=9, fontweight='bold')
ax.set_ylabel('MAPE (%)')
ax.set_title('MAPE by Feature Drop Scenario', fontweight='bold')
ax.axhline(baseline_mape, color='green', ls='--', alpha=0.5, label='Baseline')
ax.legend(fontsize=9)
plt.xticks(fontsize=8)

plt.suptitle('Part B: Missing Feature Test', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---

## Part C: K-Means Comparison

HDBSCAN is density-based and automatic. K-Means requires choosing `k` upfront and assumes spherical clusters. We compare them to verify that HDBSCAN is the right choice.

We test K-Means with k = 20, 30, 40 and compare against our best HDBSCAN result.

In [ ]:
# ============================================================
# Part C: Run K-Means with different k values
# ============================================================
KMEANS_K_VALUES = [20, 30, 40]

kmeans_results = []

for k in KMEANS_K_VALUES:
    print(f'\nRunning K-Means with k = {k}...')
    t_start = time.time()
    
    res = run_pipeline(
        df, true_fmv,
        cluster_method='kmeans',
        kmeans_k=k,
        label=f'KMeans(k={k})'
    )
    
    t_total = time.time() - t_start
    
    kmeans_results.append({
        'Method': f'K-Means (k={k})',
        'n_clusters': k,
        'noise_pct': 0.0,  # K-Means has no noise concept
        'silhouette': res['silhouette'],
        'mean_rent_cv': res['mean_rent_cv'],
        'core_pct': res['core_pct'] * 100,
        'mape': res['mape'],
        'mae': res['mae'],
        'r2': res['r2'],
        'bias': res['bias'],
        'time_s': t_total,
    })
    
    print(f'  Silhouette: {res["silhouette"]:.3f}, Rent CV: {res["mean_rent_cv"]:.3f}')
    print(f'  MAPE: {res["mape"]:.1f}%, MAE: ${res["mae"]:,.0f}, R2: {res["r2"]:.3f}')

kmeans_df = pd.DataFrame(kmeans_results)

In [ ]:
# ============================================================
# Part C: HDBSCAN vs K-Means Comparison
# ============================================================

# Best HDBSCAN result
best_hdbscan_idx = gran_df['mape'].idxmin()
best_hdbscan = gran_df.loc[best_hdbscan_idx]

comparison_rows = []

# Add best HDBSCAN
comparison_rows.append({
    'Method': f'HDBSCAN (mcs={int(best_hdbscan["min_cluster_size"])})',
    'Clusters': int(best_hdbscan['n_clusters']),
    'Noise %': f'{best_hdbscan["noise_pct"]:.1f}%',
    'Silhouette': f'{best_hdbscan["silhouette"]:.3f}',
    'Rent CV': f'{best_hdbscan["mean_rent_cv"]:.3f}',
    'MAPE': f'{best_hdbscan["mape"]:.1f}%',
    'MAE ($)': f'${best_hdbscan["mae"]:,.0f}',
    'R2': f'{best_hdbscan["r2"]:.3f}',
})

# Add all HDBSCAN mcs=20 (default)
default_hdbscan = gran_df.loc[gran_df['min_cluster_size'] == 20].iloc[0]
comparison_rows.append({
    'Method': 'HDBSCAN (mcs=20, default)',
    'Clusters': int(default_hdbscan['n_clusters']),
    'Noise %': f'{default_hdbscan["noise_pct"]:.1f}%',
    'Silhouette': f'{default_hdbscan["silhouette"]:.3f}',
    'Rent CV': f'{default_hdbscan["mean_rent_cv"]:.3f}',
    'MAPE': f'{default_hdbscan["mape"]:.1f}%',
    'MAE ($)': f'${default_hdbscan["mae"]:,.0f}',
    'R2': f'{default_hdbscan["r2"]:.3f}',
})

# Add K-Means results
for _, row in kmeans_df.iterrows():
    comparison_rows.append({
        'Method': row['Method'],
        'Clusters': int(row['n_clusters']),
        'Noise %': '0.0%',
        'Silhouette': f'{row["silhouette"]:.3f}',
        'Rent CV': f'{row["mean_rent_cv"]:.3f}',
        'MAPE': f'{row["mape"]:.1f}%',
        'MAE ($)': f'${row["mae"]:,.0f}',
        'R2': f'{row["r2"]:.3f}',
    })

comparison_df = pd.DataFrame(comparison_rows)

print('='*90)
print('HDBSCAN vs K-MEANS COMPARISON')
print('='*90)
print(comparison_df.to_string(index=False))

In [ ]:
# ============================================================
# Part C Plot: HDBSCAN vs K-Means
# ============================================================
fig, axes = plt.subplots(1, 3, figsize=(18, 5.5))

# Combine all methods for plotting
all_methods = []
all_methods.append(('HDBSCAN\n(best)', best_hdbscan['silhouette'],
                    best_hdbscan['mean_rent_cv'], best_hdbscan['mape']))
all_methods.append(('HDBSCAN\n(mcs=20)', default_hdbscan['silhouette'],
                    default_hdbscan['mean_rent_cv'], default_hdbscan['mape']))
for _, row in kmeans_df.iterrows():
    all_methods.append((f'KMeans\n(k={int(row["n_clusters"])})',
                       row['silhouette'], row['mean_rent_cv'], row['mape']))

names = [m[0] for m in all_methods]
sils = [m[1] for m in all_methods]
cvs = [m[2] for m in all_methods]
mapes = [m[3] for m in all_methods]

colors_method = ['#2ecc71', '#27ae60'] + ['#3498db'] * len(KMEANS_K_VALUES)

# (0) Silhouette
ax = axes[0]
bars = ax.bar(names, sils, color=colors_method, edgecolor='white', width=0.6)
for bar, val in zip(bars, sils):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
            f'{val:.3f}', ha='center', fontsize=9, fontweight='bold')
ax.set_ylabel('Silhouette Score')
ax.set_title('Silhouette Score Comparison', fontweight='bold')
plt.sca(ax)
plt.xticks(fontsize=8)

# (1) Within-cluster rent CV
ax = axes[1]
bars = ax.bar(names, cvs, color=colors_method, edgecolor='white', width=0.6)
for bar, val in zip(bars, cvs):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
            f'{val:.3f}', ha='center', fontsize=9, fontweight='bold')
ax.set_ylabel('Within-Cluster Rent CV')
ax.set_title('Cluster Homogeneity (lower = better)', fontweight='bold')
plt.sca(ax)
plt.xticks(fontsize=8)

# (2) MAPE
ax = axes[2]
bars = ax.bar(names, mapes, color=colors_method, edgecolor='white', width=0.6)
for bar, val in zip(bars, mapes):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2,
            f'{val:.1f}%', ha='center', fontsize=9, fontweight='bold')
ax.set_ylabel('MAPE (%)')
ax.set_title('Final MAPE Comparison', fontweight='bold')
ax.axhline(15, color='green', ls='--', alpha=0.4, label='15% target')
ax.legend(fontsize=9)
plt.sca(ax)
plt.xticks(fontsize=8)

plt.suptitle('Part C: HDBSCAN vs K-Means', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Verdict
best_km_mape = kmeans_df['mape'].min()
best_hdb_mape = best_hdbscan['mape']
if best_hdb_mape <= best_km_mape:
    print(f'\nVerdict: HDBSCAN wins (MAPE {best_hdb_mape:.1f}% vs K-Means best {best_km_mape:.1f}%)')
else:
    print(f'\nVerdict: K-Means wins (MAPE {best_km_mape:.1f}% vs HDBSCAN best {best_hdb_mape:.1f}%)')

---

## 5. Comprehensive Summary Table

All scenarios, all metrics, all recommendations.

In [ ]:
# ============================================================
# Build the master summary table
# ============================================================
summary_rows = []

# Part A: Granularity results
for _, row in gran_df.iterrows():
    summary_rows.append({
        'Part': 'A - Granularity',
        'Scenario': f'HDBSCAN mcs={int(row["min_cluster_size"])}',
        'Clusters': int(row['n_clusters']),
        'Silhouette': round(row['silhouette'], 3),
        'Rent CV': round(row['mean_rent_cv'], 3),
        'MAPE (%)': round(row['mape'], 1),
        'MAE ($)': round(row['mae'], 0),
        'R2': round(row['r2'], 3),
        'Bias ($)': round(row['bias'], 0),
    })

# Part B: Missing feature results
for _, row in drop_df.iterrows():
    summary_rows.append({
        'Part': 'B - Features',
        'Scenario': row['Scenario'],
        'Clusters': int(row['Clusters']),
        'Silhouette': round(row['Silhouette'], 3),
        'Rent CV': round(row['Rent CV'], 3),
        'MAPE (%)': round(row['MAPE'], 1),
        'MAE ($)': round(row['MAE'], 0),
        'R2': round(row['R2'], 3),
        'Bias ($)': round(row['Bias ($)'], 0),
    })

# Part C: K-Means results
for _, row in kmeans_df.iterrows():
    summary_rows.append({
        'Part': 'C - K-Means',
        'Scenario': row['Method'],
        'Clusters': int(row['n_clusters']),
        'Silhouette': round(row['silhouette'], 3),
        'Rent CV': round(row['mean_rent_cv'], 3),
        'MAPE (%)': round(row['mape'], 1),
        'MAE ($)': round(row['mae'], 0),
        'R2': round(row['r2'], 3),
        'Bias ($)': round(row['bias'], 0),
    })

summary_df = pd.DataFrame(summary_rows)

print('='*110)
print('SIMULATION 04 -- COMPREHENSIVE SUMMARY TABLE')
print('='*110)
print(summary_df.to_string(index=False))

In [ ]:
# ============================================================
# Final visualization: All parts together
# ============================================================
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# --- (0) Part A: MAPE curve ---
ax = axes[0]
ax.plot(gran_df['min_cluster_size'], gran_df['mape'], 'ro-', ms=10, lw=2)
best_mcs = gran_df.loc[gran_df['mape'].idxmin(), 'min_cluster_size']
best_mape_a = gran_df['mape'].min()
ax.scatter([best_mcs], [best_mape_a], s=250, c='gold', edgecolors='black',
           zorder=5, marker='*')
ax.annotate(f'Best: mcs={int(best_mcs)}\nMAPE={best_mape_a:.1f}%',
            (best_mcs, best_mape_a), textcoords='offset points',
            xytext=(30, -20), fontsize=10, fontweight='bold',
            arrowprops=dict(arrowstyle='->', color='black'))
ax.set_xlabel('min_cluster_size')
ax.set_ylabel('MAPE (%)')
ax.set_title('Part A: Cluster Granularity\n(HDBSCAN min_cluster_size sweep)',
             fontweight='bold')
ax.set_xscale('log')
ax.axhline(15, color='green', ls='--', alpha=0.4)

# --- (1) Part B: Feature importance bar ---
ax = axes[1]
non_bl = drop_df[drop_df['Scenario'] != 'Baseline (all features)'].copy()
non_bl = non_bl.sort_values('MAPE Increase')
short_labels_b = [
    s.replace('Drop ', '').replace('ALL ', '')
    for s in non_bl['Scenario']
]
bar_colors_b = ['#e74c3c' if x > 1.5 else '#f39c12' if x > 0.3 else '#2ecc71'
                for x in non_bl['MAPE Increase']]
bars = ax.barh(range(len(non_bl)), non_bl['MAPE Increase'],
               color=bar_colors_b, edgecolor='white', height=0.6)
ax.set_yticks(range(len(non_bl)))
ax.set_yticklabels(short_labels_b, fontsize=9)
for i, val in enumerate(non_bl['MAPE Increase']):
    sign = '+' if val >= 0 else ''
    ax.text(max(val, 0) + 0.05, i, f'{sign}{val:.1f}pp', va='center', fontsize=9)
ax.set_xlabel('MAPE Increase (pp)')
ax.set_title('Part B: Feature Importance\n(MAPE increase when dropped)', fontweight='bold')
ax.axvline(0, color='black', ls='-', lw=0.5)

# --- (2) Part C: HDBSCAN vs K-Means ---
ax = axes[2]
methods_c = ['HDBSCAN\n(best)', 'HDBSCAN\n(default)']
mapes_c = [best_hdbscan['mape'], default_hdbscan['mape']]
colors_c = ['#2ecc71', '#27ae60']
for _, row in kmeans_df.iterrows():
    methods_c.append(f'K-Means\n(k={int(row["n_clusters"])})')
    mapes_c.append(row['mape'])
    colors_c.append('#3498db')

bars = ax.bar(methods_c, mapes_c, color=colors_c, edgecolor='white', width=0.6)
for bar, val in zip(bars, mapes_c):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2,
            f'{val:.1f}%', ha='center', fontsize=9, fontweight='bold')
ax.set_ylabel('MAPE (%)')
ax.set_title('Part C: HDBSCAN vs K-Means\n(Final MAPE comparison)', fontweight='bold')
ax.axhline(15, color='green', ls='--', alpha=0.4)
plt.sca(ax)
plt.xticks(fontsize=8)

plt.suptitle('Simulation 04 -- Cluster Quality Stress Test: All Results',
             fontsize=15, fontweight='bold', y=1.03)
plt.tight_layout()
plt.show()

## 6. Conclusions & Recommendations

In [ ]:
# ============================================================
# Print final conclusions
# ============================================================
print('='*80)
print('SIMULATION 04 -- CONCLUSIONS & RECOMMENDATIONS')
print('='*80)

# Part A conclusions
opt_a = gran_df.loc[gran_df['mape'].idxmin()]
worst_a = gran_df.loc[gran_df['mape'].idxmax()]
print(f'\n1. CLUSTER GRANULARITY (Part A):')
print(f'   Best: min_cluster_size={int(opt_a["min_cluster_size"])} '
      f'(MAPE={opt_a["mape"]:.1f}%, {int(opt_a["n_clusters"])} clusters, '
      f'Sil={opt_a["silhouette"]:.3f})')
print(f'   Worst: min_cluster_size={int(worst_a["min_cluster_size"])} '
      f'(MAPE={worst_a["mape"]:.1f}%)')
print(f'   MAPE spread: {worst_a["mape"] - opt_a["mape"]:.1f} percentage points')
print(f'   Recommendation: Use min_cluster_size in [{int(opt_a["min_cluster_size"])}, '
      f'{int(opt_a["min_cluster_size"]) * 2}] range')

# Part B conclusions
print(f'\n2. FEATURE IMPORTANCE (Part B):')
sorted_drops = drop_df[drop_df['Scenario'] != 'Baseline (all features)'].sort_values(
    'MAPE Increase', ascending=False
)
for _, row in sorted_drops.iterrows():
    impact = 'HIGH' if row['MAPE Increase'] > 1.5 else 'MEDIUM' if row['MAPE Increase'] > 0.3 else 'LOW'
    print(f'   [{impact}] {row["Scenario"]}: +{row["MAPE Increase"]:.1f}pp MAPE increase')

most_critical = sorted_drops.iloc[0]
print(f'\n   Most critical feature/group: {most_critical["Scenario"]} '
      f'(+{most_critical["MAPE Increase"]:.1f}pp)')

# Part C conclusions
print(f'\n3. HDBSCAN vs K-MEANS (Part C):')
best_km_mape = kmeans_df['mape'].min()
best_km_row = kmeans_df.loc[kmeans_df['mape'].idxmin()]
best_hdb_mape = opt_a['mape']

if best_hdb_mape <= best_km_mape:
    print(f'   HDBSCAN outperforms K-Means:')
    print(f'     HDBSCAN best: MAPE={best_hdb_mape:.1f}%')
    print(f'     K-Means best: MAPE={best_km_mape:.1f}% ({best_km_row["Method"]})')
    print(f'     Advantage: {best_km_mape - best_hdb_mape:.1f}pp lower MAPE')
else:
    print(f'   K-Means outperforms HDBSCAN (unexpected!):')
    print(f'     K-Means best: MAPE={best_km_mape:.1f}% ({best_km_row["Method"]})')
    print(f'     HDBSCAN best: MAPE={best_hdb_mape:.1f}%')

print(f'\n   Additional HDBSCAN advantages:')
print(f'     - Automatic cluster count (no need to tune k)')
print(f'     - Handles noise points explicitly')
print(f'     - Adapts to varying density (urban vs rural)')

# Overall recommendations
print(f'\n4. OVERALL RECOMMENDATIONS:')
print(f'   a) Use HDBSCAN with min_cluster_size={int(opt_a["min_cluster_size"])} (optimal from Part A)')
print(f'   b) Ensure census_median_home_value is always available (highest-impact feature)')
print(f'   c) Include Z_NETWORK features in the supervised model for best accuracy')
print(f'   d) Monitor silhouette score: target > 0.30 for acceptable cluster quality')
print(f'   e) Monitor within-cluster rent CV: target < 0.30 for homogeneous clusters')
print(f'   f) If HDBSCAN produces <5 clusters, consider reducing min_cluster_size')

print('\n' + '='*80)
print('Simulation 04 complete.')
print('='*80)